# 6장 - 메타클래스와 애트리뷰트


- 파이썬의 특성을 열거할 때 메타클래스를 자주 언급하는데, 실제로 메타클래스가 어떤 목적으로 쓰이는지 이해하는 프로그래머는 거의 없다.
- <font color='blue'>메타클래스</font> 라는 이름은 어렴풋이 이 개념이 클래스를 넘어서는 것임을 암시한다.
- 간단히 말해, 메타클래스를 사용하면 파이썬의 class 문을 가로채서 클래스가 정의될 때마다 특별한 동작을 제공할 수 있다.
<br>
<br>
- 메타클래스처럼 신비롭고 강력한 파이썬 기능으로는 "동적으로 애트리뷰트 접근을 커스텀화해주는 내장 기능" 을 들 수 있다.
- 파이썬의 객체지향적인 요소와 방금 말한 두 기능이 함께 어우러지면 간단한 클래스를 복잡한 클래스로 쉽게 변환할 수 있다.
<br>
<br>
- 하지만, 이런 강력함에는 많은 함정이 뒤따른다.
- 동적인 애트리뷰트로 객체를 오버라이드하면 예기치 못한 부작용이 생길 수 있다.
- <font color='blue'>최소 놀람의 법칙(rule of least surprise)을 따르고 잘 정해진 관용어로만 이런 기능을 사용하는 것이 중요하다.</font>

### BETTER WAY 44 - 세터와 게터 메서드 대신 평범한 애트리뷰트를 사용하라

- 다른 언어를 사용하다 파이썬을 접한 프로그래머들은 클래스에 getter 나 setter 를 명시적으로 정의하곤 한다.

In [1]:
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms
        
r0 = OldResistor(50e3)
print(r0.get_ohms())
r0.set_ohms(10e3)
print(r0.get_ohms())

50000.0
10000.0


- 세터와 게터는 사용하기는 쉽지만, 이런 코드는 파이썬답지 않다.
- 대신 다음 코드와 같이 항상 단순한 공개 애트리뷰트로부터 구현을 시작하라

In [2]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resistor(50e3)
r1.ohms = 10e3

- 나중에 애트리뷰트가 설정될 때 특별한 기능을 수행해야 한다면, 애트리뷰트를 @property 데코레이터와 대응하는 setter 애트리뷰트로 옮겨갈 수 있다.

In [3]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
r2 = VoltageResistance(1e3)
print(f'이전: {r2.current:.2f} 암페어')
r2.voltage = 10
print(f'이후: {r2.current:.2f} 암페어')

이전: 0.00 암페어
이후: 0.01 암페어


- 프로퍼티에 setter 를 지정하면 타입을 검사하거나 클래스 프로퍼티에 전달된 값에 대한 검증을 수행할 수 있다.

In [4]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'저항은 0보다 큰 값이어야 합니다. 실제 값: {ohms}')
        self._ohms = ohms
        
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 저항은 0보다 큰 값이어야 합니다. 실제 값: 0

### BETTER WAY 45 - 애트리뷰트를 리팩터링하는 대신 @property 를 사용하라

- 내장 @property 데코레이터를 사용하면, 겉으로는 단순한 애트리뷰트처럼 보이지만 실제로는 지능적인 로직을 수행하는 애트리뷰트를 정의할 수 있다.
<br>
<br>
- @property 의 고급 활용법이자 흔히 사용하는 기법으로는, 간단한 수치 애트리뷰트를 그때그때 요청에 따라 계산해 제공하도록 바꾸는 것을 들 수 있다.
- 이 기법은 기존 클래스를 호출하는 코드를 전혀 바꾸지 않고도 클래스 애트리뷰트의 기존 동작을 변경할 수 있기 때문에 아주 유용하다.
- 특히 이 방법은 이 클래스를 호출하는 코드 중에 여러분이 제어할 수 없는 코드가 더 많은 경우 더욱 유용하다.
<br>
<br>
- @property 는 인터페이스를 점차 개선해나가는 과정에서 중간중간 필요한 기능을 제공하는 수단으로로 유용하다.

In [6]:
# 예) 일반 파이썬 객체를 사용해 리키 버킷(leaky bucket) 흐름 제어 알고리즘을 구현한다고 하자
# 다음 코드의 Bucket 클래스는 남은 가용 용량(quota)과 이 가용 용량의 잔존 시간을 표현

from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0
        
    def __repr__(self):
        return f'Bucket(quota={self.quota})'
    
# 리키 버킷 알고리즘은 시간을 일정한 간격(주기)으로 구분하고, 
# 가용 용량을 소비할 때마다 시간을 검사해서 주기가 달라질 경우에는 이전 주기에 미사용한 가용 용량이 새로운 주기로 넘어오지 못하게 막는다.
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = 0
    bucket.quota += amount
        
# 어떤 작업을 하고 싶을 땐 먼저 리키 버킷으로부터 자신의 작업에 필요한 용량을 할당받아야 한다.
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False # 새 주기가 시작됐는데 아직 버킷 할당량이 재설정되지 않았다.
    if bucket.quota - amount < 0:
        return False # 버킷의 가용 용량이 충분하지 못하다.
    else:
        bucket.quota -= amount
        return True # 버킷의 가용 용량이 충분하므로 필요한 분량을 사용한다.
        
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [7]:
deduct(bucket, 60)
print(bucket)
deduct(bucket, 60)
print(bucket)

Bucket(quota=40)
Bucket(quota=40)


In [11]:
# 이 구현의 문제점은 버킷이 시작할 때 가용 용량이 얼마인지 알 수 없다는 것이다.
# 이를 해결하기 위해 이번 주기에 재설정된 가용 용량인 max_quota 와 이번 주기에 버킷에서 소비한 용량의 합계인 quota_consumed 를 추적하도록 클래스를 변경할 수 있다.

class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, quota_consumed={self.quota_consumed})')
    
    # 원래의 Bucket 클래스와 인터페이스를 동일하게 제공하기 위해 @property 데코레이터가 붙은 메서드를 사용해
    # 클래스의 두 애트리뷰트(max_quota와 quota_consumed) 에서 현재 가용 용량 수준을 그때그때 계산하게 한다.
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    # fill 과 deduct 함수가 quota 애트리뷰트에 값을 할당할 때는 NewBucket 클래스의 현재 사용 방식에 맞춰 특별한 동작을 수행해야 한다.
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # 새로운 주기가 되고 가용 용량을 재설정하는 경우
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # 새로운 주기가 되고, 가용 용량을 추가하는 경우
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # 어떤 주기 안에서 가용 용량을 소비하는 경우
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta
            
bucket = NewBucket(60)
print('최초', bucket)
fill(bucket, 100)
print('보충 후', bucket)

최초 NewBucket(max_quota=0, quota_consumed=0)
보충 후 NewBucket(max_quota=100, quota_consumed=0)


In [12]:
if deduct(bucket, 99):
    print('99 용량 사용')
else:
    print('가용 용량이 작아서 99 용량을 처리할 수 없음')
print('사용 후', bucket)

if deduct(bucket, 10):
    print('10 용량 사용')
else:
    print('가용 용량이 작아서 10 용량을 처리할 수 없음')
print('여전히', bucket)

99 용량 사용
사용 후 NewBucket(max_quota=100, quota_consumed=99)
가용 용량이 작아서 10 용량을 처리할 수 없음
여전히 NewBucket(max_quota=100, quota_consumed=99)


- @property 를 사용하면 데이터 모델을 점진적으로 개선할 수 있으므로 필자는 특히 @property를 좋아한다.

### BETTER WAY 46 - 재사용 가능한 @property 메서드를 만들려면 디스크립터를 사용하라

- @property 내장 기능의 가장 큰 문제점은 재사용성이다.
- @property 가 데코레이션하는 메서드를 같은 클래스에 속하는 여러 애트리뷰트로 사용할 수는 없다.
- 서로 무관한 클래스 사이에서 @property 데코레이터를 적용한 메서드를 재사용할 수도 없다.

In [1]:
# 예) 학생의 숙제 점수가 백분율 값인지 검증하고 싶다

class Homework:
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('점수는 0과 100 사이입니다.')
        self._grade = value
        
# @property 를 사용하면 이 클래스를 쉽게 사용할 수 있다.
galileo = Homework()
galileo.grade = 95

# 이 학생에게 시험 점수를 부여하고 싶다고 하자
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('점수는 0과 100 사이입니다.')
            
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
        
    @property
    def math_grade(self):
        return self._writing_grade
    
    @math_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
        
    # 이런 식으로 과목에 대해서 계속 확장하면, 지겹다. 게다가 이런 접근 방법은 일반적이지도 않다.
    # 검증 대상이 늘어나면 @property 와 검증 대상_grade 세터 메서드를 번거롭게 다시 작성해야 한다.
    
    '''
    이런 경우 파이썬에서 적용할 수 있는 더 나은 방법은 디스크립터를 사용하는 것이다.
    '''

- <font color='blue'>디스크립터 프로토콜</font>은 파이썬 언어에서 애트리뷰트 접근을 해석하는 방법을 정의한다.
- 디스크립터 클래스는 \_\_get__ 과 \_\_set__ 메서드를 제공하고, 이 두 메서드를 사용하면 별다른 준비 코드 없이도 원하는 점수 검증 동작을 재사용할 수 있다.

In [2]:
class Grade:
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('점수는 0과 100 사이입니다.')
        self._value = value
    
class Exam:
    # 클래스 애트리뷰트
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

'''
불행히도 이 구현은 틀렸고, 잘못 동작한다.
인스턴스가 하나인 경우에는 예상대로 동작하지만, 여러 인스턴스 객체에 대해서 시도하면 예기치 못한 동작을 볼 수 있다.
'''

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('쓰기', first_exam.writing_grade)
print('과학', first_exam.science_grade)

쓰기 82
과학 99


In [3]:
second_exam = Exam()
second_exam.writing_grade = 75
print('두 번째 쓰기', second_exam.writing_grade)
print('첫 번째 쓰기', first_exam.writing_grade) # 82 이어야 하는데, 75 가 나온다!!!!!!!!!!!!!

두 번째 쓰기 75
첫 번째 쓰기 75


In [4]:
'''
문제는 writing_grde 클래스 애트리뷰트로 한 Grade 인스턴스를 모든 Exam 인스턴스가 공유한다는 점이다.
프로그램이 실행되는 동안 Exam 클래스가 처음 정의될 때, 이 애트리뷰트에 대한 Grade 인스턴스가 단 한 번만 생성된다.
Exam 인스턴스가 생성될때마다 매번 Grade 인스턴스가 생성되지는 않는다.             오호라......
'''
'''
이를 해결하려면 Grade 클래스가 각각의 유일한 Exam 인스턴스에 대해 따로 값을 추적하게 해야 한다.
인스턴스 상태를 딕셔너리에 저장하면 이런 구현이 가능하다.
'''
class Grade:
    def __init__(self):
        self._values = {}
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('점수는 0과 100 사이입니다.')
        self._values[instance] = value
        
class Exam:
    # 클래스 애트리뷰트
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('쓰기', first_exam.writing_grade)
print('과학', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('두 번째 쓰기', second_exam.writing_grade)
print('첫 번째 쓰기', first_exam.writing_grade)

쓰기 82
과학 99
두 번째 쓰기 75
첫 번째 쓰기 82


In [5]:
'''
위 구현은 간단하지만, 함정이 존재한다.
바로 메모리를 누수 시킨다는 점이다.
_values 딕셔너리는 프로그램이 실행되는 동안 모든 Exam 인스턴스에 대한 참조를 저장하고 있으므로,
인스턴스에 대한 참조 카운터가 절대로 0 이 될 수 없고, 따라서 가비지 컬렉터가 인스턴스 메모리를 절대 재활용하지 못한다.
'''
'''
이 문제를 해결하기 위해 파이썬 weakref 내장 모듈을 사용할 수 있다.
이 모듈은 WeakKeyDictioinary 라는 특별한 클래스를 제공한다.
이는 딕셔너리에 객체를 저장할 때 일반적인 강한 참조 대신에 약한 참조를 사용한다는 점이다.
파이썬 가비지 컬렉터에 의해 약한 참조로만 참조되는 객체가 사용중인 메모리를 언제든지 재활용할 수 있다.
'''
from weakref import WeakKeyDictionary
class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('점수는 0과 100 사이입니다.')
        self._values[instance] = value
        
class Exam:
    # 클래스 애트리뷰트
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('쓰기', first_exam.writing_grade)
print('과학', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('두 번째 쓰기', second_exam.writing_grade)
print('첫 번째 쓰기', first_exam.writing_grade)

쓰기 82
과학 99
두 번째 쓰기 75
첫 번째 쓰기 82


### BETTER WAY 47 - 지연 계산 애트리뷰트가 필요하면 \_\_getattr__, \_\_getattribute__, \_\_setattr__ 을 사용하라

- 파이썬 object 훅을 사용하면 시스템을 서로 접합하는 제너릭 코드를 쉽게 작성할 수 있다.

In [6]:
# 예를 들어
# 데이터베이스 레코드를 파이썬 객체로 표현하고 싶다고 하자.

# 데이터베이스에는 이미 스키마 집합이 있다.
# 우리가 만들 레코드에 대응하는 코드도 데이터베이스 스키마가 어떤 모습인지 알아야 한다.
# 하지만, 파이썬에서 데이터베이스와 파이썬 객체를 연결해주는 코드가 특정 스키마만 표현할 필요는 없다.

# 스키마를 표현하는 클래스는 더 일반적으로 만들 수 있다.

# How?
# 파이썬에서는 __getattr__ 이라는 특별 메서드를 사용해 이런 동적 기능을 활용할 수 있다.
# 어떤 클래스안에 __getattr__ 메서드 정의가 있으면, 이 객체의 인스턴스 딕셔너리에서 찾을 수 없는 애트리뷰트 접근할때마다 __getattr__ 이 호출된다.

class LazyRecord:
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = f'{name}를 위한 값'
        setattr(self, name, value)
        return value
    
data = LazyRecord()
print('이전:', data.__dict__)
print('foo:', data.foo) # 존재하지 않는 애트리뷰트 접근. 따라서 __getattr__ 이 호출되고 이 메서드는 인스턴스 딕셔너리를 변경한다.
print('이후:', data.__dict__)

이전: {'exists': 5}
foo: foo를 위한 값
이후: {'exists': 5, 'foo': 'foo를 위한 값'}


### BETTER WAY 48 - \_\_init_subclass__를 사용해 하위 클래스를 검증하라

- 메타클래스의 가장 간단한 활용법 중 하나는 어떤 클래스가 제대로 구현됬는지 검증하는 것이다.
- 복잡한 클래스 계층을 설계할 때, 어떤 스타일을 강제로 지키도록 만들거나, 메서드를 오버라이드하도록 요청하거나, 클래스 애트리뷰트 사이에 엄격한 관계를 가지도록 요구할 수 있다.
- 메타클래스는 이런 목적을 달성할 수 있다.
- 새로운 하위 클래스가 정의될 때마다 이런 검증 코드를 수행하는 신뢰성 있는 방법을 제공하기 때문이다.

 - 메타클래스가 어떻게 작동하는지 이해하는 것이 중요하다.
 - 메타클래스는 type을 상속해 정의한다.
 - 기본적인 경우 메타클래스는 \_\_new__ 메서드를 통해 자신과 연관된 클래스의 내용을 받는다.

In [5]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(f'* 실행: {name}의 메타 {meta}.__new__')
        print('기반 클래스들:', bases)
        print(class_dict)
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass
    
class MySubclass(MyClass):
    other = 567
    
    def bar(self):
        pass
    
a = MyClass()

* 실행: MyClass의 메타 <class '__main__.Meta'>.__new__
기반 클래스들: ()
{'__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123, 'foo': <function MyClass.foo at 0x000002EC86DCCC10>}
* 실행: MySubclass의 메타 <class '__main__.Meta'>.__new__
기반 클래스들: (<class '__main__.MyClass'>,)
{'__module__': '__main__', '__qualname__': 'MySubclass', 'other': 567, 'bar': <function MySubclass.bar at 0x000002EC86DCC3A0>}


In [6]:
b = MySubclass()

- 메타클래스는 클래스 이름, 클래스가 상속하는 부모 클래스들, class 의 본문에 정의된 모든 클래스 애트리뷰트에 접근할 수 있다.
- 모든 클래스는 object 를 상속하기 때문에 메타클래스가 받는 부모 클래스의 튜플 안에는 object 가 명시적으로 들어 있지 않다.

In [11]:
# 연관된 클래스가 정의되기 전에 이 클래스의 모든 파라미터를 검증하려면 Meta.__new__에 기능을 추가해야 한다.

# 예) 다각형을 표현하는 타입

class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Polygon 클래스의 하위 클래스만 검증한다.
        if bases:
            if class_dict['sides'] < 3:
                raise ValueError('다각형 변은 3개 이상이어야 함')
        return type.__new__(meta, name, bases, class_dict)
    
class Polygon(metaclass=ValidatePolygon):
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Triangle(Polygon):
    sides = 3
    
class Rectangle(Polygon):
    sides = 4
    
class Nonagon(Polygon):
    sides = 9

assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260

In [12]:
print('class 이전')

class Line(Polygon):
    print('sides 이전')
    sides = 2
    print('sides 이후')
    
print('class 이후')

class 이전
sides 이전
sides 이후


ValueError: 다각형 변은 3개 이상이어야 함

- 파이썬에게 이런 기본적인 작업을 시키기 위해 너무 복잡한 코드를 작성해야 하는 것처럼 보인다.
- 다행히 파이썬 3.6 에는 메타클래스를 정의하지 않고 같은 동작을 구현할 수 있는 더 단순한 구문 (\_\_init_subclass__ 특별 클래스 메서드를 정의하는 방식)이 추가되었다.

In [26]:
class BetterPolygon:
    sides = None # 이 하위 클래스에서 이 애트리뷰트 값을 지정해야 함
        
    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError('다각형 변은 3개 이상이어야 함')
        
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Hexagon(BetterPolygon):
    sides = 6
    
assert Hexagon.interior_angles() == 720

In [27]:
print('class 이전')

class Line(BetterPolygon):
    print('sides 이전')
    sides = 1
    print('sides 이후')
    
print('class 이후')

class 이전
sides 이전
sides 이후


ValueError: 다각형 변은 3개 이상이어야 함

### BETTER WAY 49 - \_\_init_subclass__를 사용해 클래스 확장을 등록하라

- 메타클래스의 다른 용례로 프로그램이 자동으로 타입을 등록하는 것이 있다.
- 간단한 식별자를 이용해 그에 해당하는 클래스를 찾는 역검색을 하고 싶을 때 이런 등록 기능이 유용하다.

In [1]:
# 예) 파이썬 object 를 JSON 으로 직렬화하는 직렬화 표현 방식을 구현한다고 하자

import json

class Serializable:
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({'args': self.args})
    
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'
        
point = Point2D(5, 3)
print('객체:', point)
print('직렬화한 값:', point.serialize())

객체: Point2D(5, 3)
직렬화한 값: {"args": [5, 3]}


In [4]:
# 이제 이 JSON 문자열을 역직렬화해서 문자열이 표현하는 Point2D 객체를 구성해야 한다.

class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])
        
class BetterPoint2D(Deserializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'
    
point = BetterPoint2D(5, 3)
print('객체:', point)
data = point.serialize()
print('직렬화한 값:', data)
data2 = point.deserialize(data)
print('역직렬화한 값:', data2)

객체: Point2D(5, 3)
직렬화한 값: {"args": [5, 3]}
역직렬화한 값: Point2D(5, 3)


In [7]:
# 이 접근 방식은 우리가 Point2D 나 BetterPoint2D 라는 타입을 미리 알고 있는 경우에만 사용할 수 있다는 문제가 있다.

# 따라서, JSON 으로 직렬화 할 클래스가 아주 많더라도 역직렬화 하는 함수는 하나만 있는 것이 이상적이다.

class BetterSerializable:
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })
        
    def __repr__(self):
        name = self.__class__.__name__
        args_str = ', '.join(str(x) for x in self.args)
        return f'{name}({args_str})'
    
registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class
    
def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

# deserialize 가 항상 제대로 동작하려면, 나중에 역직렬화할 모든 클래스에서 register_class 를 호출해야 한다.

class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
register_class(EvenBetterPoint2D) # 반드시 호출

# 이후 테스트
point = EvenBetterPoint2D(5, 3)
print('객체:', point)
data = point.serialize()
print('직렬화한 값:', data)
data2 = deserialize(data)
print('역직렬화한 값:', data2)

객체: EvenBetterPoint2D(5, 3)
직렬화한 값: {"class": "EvenBetterPoint2D", "args": [5, 3]}
역직렬화한 값: EvenBetterPoint2D(5, 3)


In [8]:
'''
이 방식의 문제점은 register_class 함수 호출을 잊어버릴 수 있다는 것이다.
'''
class Point3D(BetterSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

# register_class 호출을 잊어버렸다!

# 이후 테스트
point = Point3D(5, 9, -4)
print('객체:', point)
data = point.serialize()
print('직렬화한 값:', data)
data2 = deserialize(data)
print('역직렬화한 값:', data2)

객체: Point3D(5, 9, -4)
직렬화한 값: {"class": "Point3D", "args": [5, 9, -4]}


KeyError: 'Point3D'

In [9]:
'''
프로그래머가 BetterSerializable 를 사용한다는 의도를 감지하고 적절한 동작을 수행해 항상 제대로 register_class 를 호출해 줄 수 있다면 어떨까?
메타클래스는 하위 클래스가 정의될 때 class 문을 가로채서 이런 동작을 수행할 수 있다.
'''
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls
    
class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass

# 이제는 RegisteredSerializable 의 하위 클래스를 정의할 때 register_class 가 호출되고 deserialize 가 항상 제대로 동작한다고 할 수 있다.

class Point3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

# 이후 테스트
point = Point3D(5, 9, -4)
print('객체:', point)
data = point.serialize()
print('직렬화한 값:', data)
data2 = deserialize(data)
print('역직렬화한 값:', data2)

객체: Point3D(5, 9, -4)
직렬화한 값: {"class": "Point3D", "args": [5, 9, -4]}
역직렬화한 값: Point3D(5, 9, -4)


In [10]:
# 더 좋은 접근 방법은 __init_subclass__ 특별 클래스 메서드를 사용하는 것이다.
class BetterRegisteredSerializable(BetterSerializable):
    def __init_subclass__(cls):
        super().__init_subclass__()
        register_class(cls)
        
class Point3D(BetterRegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

# 이후 테스트
point = Point3D(5, 9, -4)
print('객체:', point)
data = point.serialize()
print('직렬화한 값:', data)
data2 = deserialize(data)
print('역직렬화한 값:', data2)

객체: Point3D(5, 9, -4)
직렬화한 값: {"class": "Point3D", "args": [5, 9, -4]}
역직렬화한 값: Point3D(5, 9, -4)


### BETTER WAY 50 - \_\_set_name__ 으로 클래스 애트리뷰트를 표시하라

- 메타클래스를 통해 사용할 수 있는 유용한 기능이 한 가지 더 있다.
- 클래스가 정의된 후 클래스가 실제로 사용되기 이전인 시점에 프로퍼티를 변경하거나 표시할 수 있는 기능이다.

In [11]:
# 예) 고객 데이터베이스의 로우(row)를 표현하는 새 클래스를 정의한다고 하자.

class Field:
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
# 컬럼 이름을 Field 디스크립터에 저장하고 나면, setattr 내장 함수를 사용 해 인스턴스별 상태를 직접 인스턴스 딕셔너리에 저장할 수 있고,
# 나중에 getattr 로 인스턴스의 상태를 읽을 수 있다.

class Customer:
    # 클래스 애트리뷰트
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')
    
# 이 클래스는 사용하기 쉽다
# 예상대로 Field 디스크립터가 __dict__ 인스턴스 딕셔너리를 변화시킨다는 사실을 확인할 수 있다.

cust = Customer()
print(f'이전: {cust.first_name} {cust.__dict__}')
cust.first_name = '유클리드'
print(f'이후: {cust.first_name} {cust.__dict__}')

이전:  {}
이후: 유클리드 {'_first_name': '유클리드'}


In [12]:
'''
하지만, 이 클래스 정의는 중복이 많아 보인다.
클래스 안에서 왼쪽에 필드 이름을 정의했는데 (first_name =), 굳이 같은 정보가 들어 있는 문자열을 Field 디스크립터에게 다시 전달 해야 할 이유가 없다.

우리는 읽을 때 왼쪽에서 부터 읽지만
파이썬이 실제로 Customer 클래스를 처리하는 순서는 이와 반대이기 때문이다.
즉, 파이썬은 Field('first_name') 을 먼저 처리하고 반환된 값을 first_name = 에 넣기 때문에,
first_name 이 자신에게 대입될 클래스의 애트리뷰트 이름을 미리 알 방법은 없다.
'''
'''
이런 중복을 줄이기 위해 메타클래스를 사용할 수 있다.
'''

class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls
    
class DatabaseRow(metaclass=Meta):
    pass
    
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
class BetterCustomer(DatabaseRow):
    # 클래스 애트리뷰트
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()
    
cust = BetterCustomer()
print(f'이전: {cust.first_name} {cust.__dict__}')
cust.first_name = '유클리드'
print(f'이후: {cust.first_name} {cust.__dict__}')

이전:  {}
이후: 유클리드 {'_first_name': '유클리드'}


In [13]:
'''
이 접근 방법의 문제점은 DatabaseRow 를 상속하는 것을 잊어버리거나 클래스 계층 구조로 인해 어쩔 수 없이 DatabaseRow 를 상속할 수 없는 경우,
문제가 발생한다.

이 문제를 해결하는 방법은 디스크립터에 __set_name__ 특별 메서드를 사용하는 것이다. (파이썬 3.6 부터 도입됨)
'''
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None
        
    def __set_name__(self, owner, name):
        # 클래스가 생성될 때 모든 스크립터에 대해 이 메서드가 호출된다.
        self.name = name
        self.internal_name = '_' + name
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
class FixedCustomer:
    # 클래스 애트리뷰트
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()
    
cust = FixedCustomer()
print(f'이전: {cust.first_name} {cust.__dict__}')
cust.first_name = '유클리드'
print(f'이후: {cust.first_name} {cust.__dict__}')

이전:  {}
이후: 유클리드 {'_first_name': '유클리드'}


### BETTER WAY 51 - 합성 가능한 클래스 확장이 필요하면 메타클래스보다는 클래스 데코레이터를 사용하라

- 메타클래스를 사용하면 클래스 생성을 다양한 방법으로 커스텀화할 수 있지만, 여전히 메타클래스로 처리할 수 없는 경우가 있다.

In [14]:
# 예를 들어, 어떤 클래스의 모든 메서드를 감싸서 메서드에 전달되는 인자, 반환 값, 발생환 예외를 모두 출력하고 싶다고 하자

from functools import wraps

def trace_func(func):
    if hasattr(func, 'tracing'): # 단 한번만 데코레이터를 적용한다.
        return func
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f'{func.__name__}({args}, {kwargs}) -> {result}')
        
    wrapper.tracing = True
    return wrapper

class TraceDict(dict):
    @trace_func
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    @trace_func
    def __setitem__(self, *args, **kwargs):
        return super().__setitem__(*args, **kwargs)
        
    @trace_func
    def __getitem__(self, *args, **kwargs):
        return super().__getitem__(*args, **kwargs)
    
# 메서드가 잘 데코레이션 되었는지 확인해보자.
trace_dict = TraceDict([('안녕', 1)])
trace_dict['거기'] = 2
trace_dict['안녕']
try:
    trace_dict['존재하지 않음']
except KeyError:
    pass

__init__(({'안녕': 1}, [('안녕', 1)]), {}) -> None
__setitem__(({'안녕': 1, '거기': 2}, '거기', 2), {}) -> None
__getitem__(({'안녕': 1, '거기': 2}, '안녕'), {}) -> 1
__getitem__(({'안녕': 1, '거기': 2}, '존재하지 않음'), {}) -> '존재하지 않음'


In [15]:
'''
잘 동작한다.
하지만, 이 코드의 문제점은 꾸미려는 모든 메서드를 @trace_func 데코레이터를 써서 재정의해야 한다는 것이다.

이 문제를 해결하는 방법은,
메타클래스를 사용해 클래스에 속한 모든 메서드를 자동으로 감싸는 것이다.
'''
import types

trace_types = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType
    )

class TraceMeta(type):
    def __new__(meta, name, bases, class_dict):
        klass = super().__new__(meta, name, bases, class_dict)
        
        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_types):
                wrapped = trace_func(value)
                setattr(klass, key, wrapped)
                
        return klass
    
class TraceDict(dict, metaclass=TraceMeta):
    pass

trace_dict = TraceDict([('안녕', 1)])
trace_dict['거기'] = 2
trace_dict['안녕']
try:
    trace_dict['존재하지 않음']
except KeyError:
    pass

__format__(({}, ''), {}) -> {}
__new__((<class '__main__.TraceDict'>, [('안녕', 1)]), {}) -> {}
__getitem__(({'안녕': 1, '거기': 2}, '안녕'), {}) -> 1
__getitem__(({'안녕': 1, '거기': 2}, '존재하지 않음'), {}) -> '존재하지 않음'


In [16]:
'''
하지만, 상위 클래스가 메타클래스를 이미 정의한 경우 오류가 발생한다.
물론 어찌어찌 해결하는 방법은 있겠지만, 메타클래스를 사용하는 접근 방식은 적용 대상 클래스에 대한 제약이 너무 많다.

이런 문제를 해결하고자 파이썬은 클래스 데코레이터를 지원한다.
클래스 데코레이터는 함수 데코레이터처럼 사용할 수 있다.
즉, 클래스 선언 앞에 @ 기호와 데코레이터 함수를 적으면 된다.
이 때 데코레이터 함수는 인자로 받은 클래스를 적절히 변경해서 재생성해야 한다.
'''

def my_class_decorator(klass):
    klass.extra_param = '안녕'
    return klass

@my_class_decorator
class MyClass:
    pass

print(MyClass)
print(MyClass.extra_param)

<class '__main__.MyClass'>
안녕


In [5]:
# 앞에서 구현한 걸 클래스 데코레이터를 이용해서 구현해 보자.

import types

trace_types = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType
    )

from functools import wraps

def trace_func(func):
    if hasattr(func, 'tracing'): # 단 한번만 데코레이터를 적용한다.
        return func
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        
    wrapper.tracing = True
    return wrapper

def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        if isinstance(value, trace_types):
            wrapped = trace_func(value)
            setattr(klass, key, wrapped)
    return klass
            
@trace
class TraceDict(dict):
    pass

trace_dict = TraceDict([('안녕', 1)])
trace_dict['거기'] = 2
trace_dict['안녕']
try:
    trace_dict['존재하지 않음']
except KeyError:
    pass

__new__((<class '__main__.TraceDict'>, [('안녕', 1)]), {}) -> {}
__getitem__(({'안녕': 1, '거기': 2}, '안녕'), {}) -> 1
__getitem__(({'안녕': 1, '거기': 2}, '존재하지 않음'), {}) -> KeyError('존재하지 않음')


- 클래스를 확장하면서 합성이 가능한 방법을 찾고 있다면, 클래스 데코레이터가 가장 적합한 도구다.